In [1]:
import warnings 
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#from plotnine import *

columns = ["name","backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
#lgb1 = pd.read_pickle("lgbInvtSim_L1.pkl")#[columns] #_L_
#ets1 = pd.read_pickle("etsInvtSim_L1.pkl")#[columns] #_L_
#Idf1      = pd.concat([lgb1,ets1])

from tqdm import tqdm
lp = {"l1" :(0,1),
      "l2" :(1,(1+3)),
      "l4" :((1+3),(1+3+3)),
      "l3" :((1+3+3),(1+3+3+10)),
      "l5" :((1+3+10+3),(1+3+10+3+7)),
      "l6" :((1+3+10+3+7),(1+3+10+3+7+9)),
      "l7" :((1+3+10+3+7+9),(1+3+10+3+7+9+21)),
      "l8" :((1+3+10+3+7+9+21),(1+3+10+3+7+9+21+30)),
      "l10" :((1+3+10+3+7+9+21+30),(1+3+10+3+7+9+21+30+3049)),
      "l9":((1+3+10+3+7+9+21+30+3049),(1+3+10+3+7+9+21+30+3049+70)),
      "l11":((1+3+10+3+7+9+21+30+70+3049),(1+3+10+3+7+9+21+30+70+3049+9147)),
      "l12":((1+3+10+3+7+9+21+30+70+3049+9147),(1+3+10+3+7+9+21+30+70+3049+9147+30490))}
''' 
def check(lp:dict, df:pd.DataFrame,col: str, gap:int = 28, back_ = True):
    asl = []
    for i in lp.keys():
        l1 = np.array(df[col][lp[i][0]*gap:lp[i][1]*gap])
        if back_:
            asl2 = []
            l2 = np.array(df['true_demand'][lp[i][0]*gap:lp[i][1]*gap])
            for j in range(len(l1)):
                a = 1 if l1[j] == 0 else 1-(l1[j]/l2[j])
                asl2.append(a)
            asl.append(np.mean(asl2))
        else:
            asl.append(l1.mean())
    return asl
'''

def check(lp:dict, df:pd.DataFrame,col: str, gap:int = 28, back_ = True):
    asl = []
    for i in lp.keys():
        l1 = np.array(df[col][lp[i][0]*gap:lp[i][1]*gap])
        l2 = np.array(df['true_demand'][lp[i][0]*gap:lp[i][1]*gap])
        if back_:
            asl2 = []
            for j in range(len(l1)):
                if l1[j] == 0 or l2[j] == 0:
                    a = 1 #if  else 
                else:
                    if (l1[j]/l2[j]) <= 1:
                        a = 1-(l1[j]/l2[j])
                    else:
                        a = 0
                asl2.append(a)
            asl.append(np.mean(asl2))
        else:
            asl.append(l1.mean())
    return asl

def levels_summary2(dff:pd.DataFrame, 
                   col_1:str, 
                   col_2:str,
                   col_3:str,
                   lp_:dict, 
                   name_:str,
                   tsl_: float,
                   senario_:str,
                   c = True , 
                   gap_:int = 28, 
                   lvls:int = 12):
    dff_                           = pd.DataFrame()
    dff_['Levels']                 = [1,2,4,3,5,6,7,8,10,9,11,12]
    dff_['Forecast_Methods']       = [name_]*lvls
    dff_['Target_Service_Level']   = [tsl_]*lvls
    dff_['Senario']                = [senario_]*lvls
    dff_['Achieved_Service_Level'] = check(lp = lp_, df = dff, col = col_1, back_ = c, gap = gap_)
    dff_['Holding_Costs']          = check(lp = lp_, df = dff, col = col_2, back_ = not c, gap = gap_)
    dff_['Backlogging_Costs']      = check(lp = lp_, df = dff, col = col_3, back_ = not c, gap = gap_)
    return dff_
def asl_chcb(lp:dict, Idf:pd.DataFrame, names, names_m, col_list3, senario:str):
    summary_list = []
    for i in tqdm(range(len(names))):
        idf = Idf[Idf['name'] == names[i]]
        for j in col_list3:
            summary_list.append(levels_summary2(lp_ = lp, 
                                            dff= idf, 
                                            name_ = names_m[i],
                                            col_1 = j[0], 
                                            col_2 = j[1], 
                                            col_3 = j[2], 
                                            tsl_ = j[3], senario_ = senario))
    return pd.concat(summary_list)

In [2]:
col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
for i in  col_list3:
    print(i)

('backlog_90', 'ch_90', 'cb_90', '90%')
('backlog_95', 'ch_95', 'cb_95', '95%')
('backlog_99', 'ch_99', 'cb_99', '99%')


## COSTS & ACHIEVED SERVICE LEVEL

## Inventory performance 1:

In [ ]:
col_list1 = ['backlog_90','backlog_95','backlog_99']
col_list2 = ['ch_90','cb_90','ch_95','cb_95','ch_99','cb_99']
LGBM      = levels_summary(lp_ = lp, dff= lgbbase3, col_1 = col_list1, col_2 = col_list2, name_ = 'LGBM')
LGBM_BU   = levels_summary(lp_ = lp, dff= lgbbu3,   col_1 = col_list1, col_2 = col_list2, name_ = 'LGBM_BU')
LGBM_TD   = levels_summary(lp_ = lp, dff= lgbtd3,   col_1 = col_list1, col_2 = col_list2, name_ = 'LGBM_TD')
LGBM_MinT = levels_summary(lp_ = lp, dff= lgbmint3, col_1 = col_list1, col_2 = col_list2, name_ = 'LGBM_MinT')

ETS       = levels_summary(lp_ = lp, dff= etsbase3, col_1 = col_list1, col_2 = col_list2, name_ = 'ETS')
ETS_BU    = levels_summary(lp_ = lp, dff= etsbu3,   col_1 = col_list1, col_2 = col_list2, name_ = 'ETS_BU')
ETS_TD    = levels_summary(lp_ = lp, dff= etstd3,   col_1 = col_list1, col_2 = col_list2, name_ = 'ETS_TD')
ETS_MinT  = levels_summary(lp_ = lp, dff= etsmint3, col_1 = col_list1, col_2 = col_list2, name_ = 'ETS_MinT')

base_invt = pd.concat([LGBM,LGBM_BU,LGBM_TD,LGBM_MinT,ETS,ETS_BU,ETS_TD,ETS_MinT])
#base_invt.to_pickle("base_ls_1.pkl")

In [ ]:
l1 = base_invt[base_invt.index == 0]
l1

## 2： Gnerated Performance for Visualization

### test:

In [6]:
import pandas as pd
columns = ["name",'true_demand',"backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
lgb1 = pd.read_pickle("lgbInvtSim_L3.pkl")[columns] #_L_
ets1 = pd.read_pickle("etsInvtSim_L3.pkl")[columns] #_L_zhme
BASE = pd.concat([lgb1,ets1])
BU   = pd.read_pickle("BUOrder_L3.pkl")[columns]
TDFP = pd.read_pickle("TDFPOrder_L3.pkl")[columns]
MinT = pd.read_pickle("MinTOrder_L3.pkl")[columns]#_L1
col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu','ets_td', 'ets_mint']
names_m   = ['LGBM','LGBM_BU','LGBM_TD','LGBM_MinT','ETS','ETS_BU','ETS_TD','ETS_MinT']
L_BU  = asl_chcb(lp=lp, Idf = BU, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BU')
L_TDFP  = asl_chcb(lp=lp, Idf = TDFP, names = names, names_m = names_m, col_list3 = col_list3, senario = 'TDFP')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:54<00:00,  6.85s/it]


In [5]:
L_BASE[L_BASE['Levels'] == 12]

,Levels,Forecast_Methods,Target_Service_Level,Senario,Achieved_Service_Level,Holding_Costs,Backlogging_Costs
11,12,LGBM,90%,BASE,0.968500,3.168955,1.355419
11,12,LGBM,95%,BASE,0.980184,3.865607,1.868114
11,12,LGBM,99%,BASE,0.991293,5.220834,4.614173
11,12,LGBM_BU,90%,BASE,0.968500,3.168955,1.355419
11,12,LGBM_BU,95%,BASE,0.980184,3.865607,1.868114
11,12,LGBM_BU,99%,BASE,0.991293,5.220834,4.614173
11,12,LGBM_TD,90%,BASE,0.976349,29.892709,1.020925
11,12,LGBM_TD,95%,BASE,0.985491,38.221115,1.379710
11,12,LGBM_TD,99%,BASE,0.993871,53.883198,3.344922
11,12,LGBM_MinT,90%,BASE,0.965108,3.095890,1.343465


###  Results:

In [8]:
# BASE:
def asl_chcb(lp:dict, Idf:pd.DataFrame, names, names_m, col_list3, senario:str):
    summary_list = []
    for i in tqdm(range(len(names))):
        idf = Idf[Idf['name'] == names[i]]
        for j in col_list3:
            summary_list.append(levels_summary2(lp_ = lp, 
                                            dff= idf, 
                                            name_ = names_m[i],
                                            col_1 = j[0], 
                                            col_2 = j[1], 
                                            col_3 = j[2], 
                                            tsl_ = j[3], senario_ = senario))
    return pd.concat(summary_list)
def collect(i: int):
    columns = ["name",'true_demand',"backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
    col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
    names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu','ets_td', 'ets_mint']
    names_m   = ['LGBM','LGBM_BU','LGBM_TD','LGBM_MinT','ETS','ETS_BU','ETS_TD','ETS_MinT']
    lgb1 = pd.read_pickle(f"lgbInvtSim_L{i}.pkl")[columns] #_L_
    ets1 = pd.read_pickle(f"etsInvtSim_L{i}.pkl")[columns] #_L_zhme
    BASE = pd.concat([lgb1,ets1])
    BU   = pd.read_pickle(f"BUOrder_L{i}.pkl")[columns]
    TDFP = pd.read_pickle(f"TDFPOrder_L{i}.pkl")[columns]
    MinT = pd.read_pickle(f"MinTOrder_L{i}.pkl")[columns]#_L1
    names     = ['lgb_base',  'ets_base']
    names_m   = ['LGBM','ETS']
    L_BASE  = asl_chcb(lp=lp, Idf = BASE, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BASE')
    
    names     = ['lgb_base', 'lgb_bu',  'ets_base', 'ets_bu']
    names_m   = ['LGBM','LGBM_BU','ETS','ETS_BU']
    L_BU    = asl_chcb(lp=lp, Idf = BU, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BU_IR')
    
    names     = ['lgb_base','lgb_td', 'ets_base','ets_td']
    names_m   = ['LGBM','LGBM_TD','ETS','ETS_TD']
    L_TDFP  = asl_chcb(lp=lp, Idf = TDFP, names = names, names_m = names_m, col_list3 = col_list3, senario = 'TD_IR')
    
    names     = ['lgb_base', 'lgb_mint', 'ets_base', 'ets_mint']
    names_m   = ['LGBM','LGBM_MinT','ETS','ETS_MinT']
    L_MinT  = asl_chcb(lp=lp, Idf = MinT, names = names, names_m = names_m, col_list3 = col_list3, senario = 'MinT_IR')
    all_ = pd.concat([L_BASE,L_BU,L_TDFP,L_MinT])
    return all_#mint_ols
for lead_time in [1,3,7,14]:
    print(f"Lead_time is {lead_time}")
    all_ = collect(i= lead_time)
    all_.to_csv(f"Base2IR_L{lead_time}.csv", index = False)

Lead_time is 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.65s/it]


Lead_time is 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.54s/it]


Lead_time is 7


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.38s/it]


Lead_time is 14


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.16s/it]
